In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from cipher import *
from ciphertext import Plaintext
from scheme import *

from algorithms import Algorithms 
from stats import Statistics
from context import set_all

In [3]:
context, ev, encoder, encryptor, decryptor = set_all(30, 150, 12)
nslots = context.params.nslots

### Load data

In [4]:
data = pd.read_csv("./stat/WA_Fn-UseC_-HR-Employee-Attrition.csv")
new_label = {"Attrition": {"Yes":1, "No":0}}
data.replace(new_label , inplace = True)

ng1 = data.Attrition.nunique()
ng2 = data.JobSatisfaction.nunique()

ntot = len(data)
print(f"Enough slots? {ntot} < {context.params.nslots}")
V1 = encryptor.encrypt(data.Attrition.values) 
V2 = encryptor.encrypt(data.JobSatisfaction.values)

Enough slots? 1470 < 4096


category 조합을 4 * 2 = 8 까지의 int로 표현

In [5]:
p_ng2 = Plaintext(np.repeat(ng2, ntot), nslots=nslots, logp=V1.logp)
ev.mult_by_plain(V1,p_ng2, inplace=True)
ev.rescale_next(V1)
ct = ev.add(V1, V2)

각 category에 해당하는 값을 count 

대소비교

In [6]:
algo = Algorithms(ev, encoder)

In [11]:
ptmp = Plaintext(np.repeat(0.5, len(ct)), nslots=ct.nslots, logp=ct.logp)
compared = algo.comp(ct, ptmp)

In [22]:
import math 
from numpy import polynomial as P
def _approx_sign(n):
    """
    Approxiate sign function in [-1,1]
    """
    p_t1 = P.Polynomial([1,0,-1])
    p_x  = P.Polynomial([0,1])

    def c_(i: int):
        return 1/4**i * math.comb(2*i,i)

    def term_(i: int):
        return c_(i) * p_x * p_t1**i

    poly = term_(0)
    for nn in range(1,n+1):
        poly += term_(nn)
    return poly

In [26]:
p_t1 = P.Polynomial([1,0,-1])

In [27]:
p_t1

Polynomial([ 1.,  0., -1.], domain=[-1,  1], window=[-1,  1])

In [28]:
fun

Polynomial([  0.        ,   3.70013809,   0.        , -12.33379364,
         0.        ,  33.30124283,   0.        , -63.43093872,
         0.        ,  86.33655548,   0.        , -84.76679993,
         0.        ,  59.77146149,   0.        , -29.60110474,
         0.        ,   9.79448318,   0.        ,  -1.9474411 ,
         0.        ,   0.17619705], domain=[-1.,  1.], window=[-1.,  1.])

In [23]:
fun = _approx_sign(10)

In [24]:
arr = np.random.rand(10) -0.5

In [25]:
fun(arr)

array([ 0.97124285,  0.72893984,  0.75952952, -0.90567384,  0.69450522,
        0.1702422 , -0.91571005, -0.68820768,  0.98661768, -0.92283003])

In [14]:
diff = ev.sub_plain(ct, ptmp)
one = Plaintext(arr=np.repeat(1., diff.nslots),
             logp = diff.logp, nslots = diff.nslots)
half = Plaintext(arr=np.repeat(.5, diff.nslots),
             logp = diff.logp, nslots = diff.nslots)
sign_plus1 = ev.add_plain(algo.sign(algo.sign(diff)), one)
result = ev.mult_by_plain(sign_plus1, half)

In [17]:
diff._arr

array([ 7.5,  1.5,  6.5, ..., -0.5, -0.5, -0.5])

In [12]:
compared._arr

array([            inf,  5.24691213e+05,             inf, ...,
       -1.88737914e-15, -1.88737914e-15, -1.88737914e-15])

In [7]:
diff = ev.sub_plain(ct, ptmp)

NameError: name 'ptmp' is not defined

In [8]:
algo.sign(diff)

NameError: name 'diff' is not defined

In [16]:
ct._arr

array([8., 2., 7., ..., 0., 0., 0.])

In [15]:
ptmp._arr

array([0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5])

In [ ]:
bin_count = []
n_unique = ng1*ng2

for i in range(n_unique):
    count = np.sum(comp(np.repeat(i+0.5, len(ct))
    bin_count.append()

In [41]:
decryptor.decrypt(ct).max()

8.0